### GibbsSampler(Dna, k, t, N)

```
GibbsSampler(Dna, k, t, N)
    randomly select k-mers Motifs = (Motif1, …, Motift) in each string from Dna
    BestMotifs ← Motifs
    for j ← 1 to N
        i ← Random(t)
        Profile ← profile matrix constructed from all strings in Motifs except for Motifi
        Motifi ← Profile-randomly generated k-mer in the i-th sequence
        if Score(Motifs) < Score(BestMotifs)
            BestMotifs ← Motifs
    return BestMotifs
```

In [ ]:
import random
def RandomMotifs(Dna, k, t):
    Motifs = []
    n = len(Dna[0])
    for i in range(t):
        start_index = random.randint(0, n-k)
        random_kmer = Dna[i][start_index:start_index+k]
        Motifs.append(random_kmer)
    return Motifs

def CountWithPseudocounts(Motifs):
    count = {}
    k = len(Motifs[0])
    for i in "ACGT":
        count[i] = []
        for j in range(k):
            count[i].append(0)
    for i in range(len(Motifs)):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    for i in 'ACGT':
        for j in range(k):
            count[i][j] += 1
    return count

def ProfileWithPseudocounts(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    Profile = CountWithPseudocounts(Motifs)
    n = 0
    for i in "ACGT":
        n = n+Profile[i][0]
    for i in "ACGT":
        for j in range(k):
            Profile[i][j] = Profile[i][j]/n
    return Profile

def Consensus(Motifs):
    consensus = ""
    k = len(Motifs[0])
    dic = CountWithPseudocounts(Motifs)
    for j in range(k):
        m = -1
        for i in 'ACGT':
            if dic[i][j] > m:
                m = dic[i][j]
                most_frequent_symbol = i
        consensus = consensus+most_frequent_symbol
    return consensus

def Score(Motifs):
    k = len(Motifs[0])
    t = len(Motifs)
    benchmark = Consensus(Motifs)
    score = 0
    for j in range(k):
        for i in range(t):
            if Motifs[i][j] != benchmark[j]:
                score += 1
    return score

def Pr(String, Profile):
    product = 1
    j = 0
    for i in String:
        product = product*(Profile[i][j])
        j += 1
    return product

def ProfileMostProbableKmer(text, k, Profile):
    n = len(text)
    m = -1
    target = ''
    for i in range(n-k+1):
        string = text[i:i+k]
        pr = Pr(string, Profile)
        if pr > m:
            target = string
            m = pr
    return target

def GibbsSampler(Dna, k, t, N):
    BestMotifs = []
    Motifs = RandomMotifs(Dna, k, t)
    BestMotifs = Motifs
    for j in range(1, N):
        i = random.randint(1, t)
        text = Motifs.pop(i-1)
        Profile = ProfileWithPseudocounts(Motifs)
        kmer = ProfileMostProbableKmer(text, k, Profile)
        Motifs.insert(i-1, kmer)
        if Score(Motifs) < Score(BestMotifs):
            BestMotifs = Motifs
    return BestMotifs


k = 8
t = 5
N = 100
Dna = ['CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA', 'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG', 'TAGTACCGAGACCGAAAGAAGTATACAGGCGT', 'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC', 'AATCCACCAGCTCCACGTGCAATGTTGGCCTA']

print(GibbsSampler(Dna, k, t, N))